# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer
import math
import json

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [2]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [3]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


### Clean portfolio

In [4]:
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [5]:
offer_type_dummy = pd.get_dummies(portfolio["offer_type"])

In [6]:
mlb = MultiLabelBinarizer()

In [7]:
channel_encoded = mlb.fit_transform(portfolio['channels'])

In [8]:
channel_dummy = pd.DataFrame(channel_encoded, columns=mlb.classes_)

In [9]:
cleaned_portfolio = pd.concat([portfolio, offer_type_dummy, channel_dummy], axis=1)

In [10]:
cleaned_portfolio.drop(columns=["offer_type", "channels"], inplace=True)

In [11]:
cleaned_portfolio.rename(columns={ "id" : "offer_id" }, inplace=True)

In [12]:
cleaned_portfolio.head()

,reward,difficulty,duration,offer_id,bogo,discount,informational,email,mobile,social,web
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,True,False,False,1,1,1,0
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,True,False,False,1,1,1,1
2,0,0,4,3f207df678b143eea3cee63160fa8bed,False,False,True,1,1,0,1
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,True,False,False,1,1,0,1
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,False,True,False,1,0,0,1


### Clean profile

In [13]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


Original dataset shape

In [14]:
profile.shape

(17000, 5)

In [15]:
cleaned_profile = profile[profile["income"].notnull()]
cleaned_profile = cleaned_profile[cleaned_profile["gender"] != 'O']

In [16]:
cleaned_profile["became_member_on"] = pd.to_datetime(cleaned_profile["became_member_on"], format="%Y%m%d")
cleaned_profile["member_days"] = (pd.Timestamp('today') - cleaned_profile["became_member_on"]).dt.days

In [17]:
lb = LabelBinarizer()

In [18]:
cleaned_profile["gender"] = lb.fit_transform(cleaned_profile["gender"])

__Cleaned dataset shape__

In [19]:
cleaned_profile.shape

(14613, 6)

In [20]:
profile.shape[0] - cleaned_profile.shape[0]

2387

In [21]:
cleaned_profile.drop(columns="became_member_on", inplace=True)

In [22]:
cleaned_profile.reset_index(drop=True, inplace=True)

In [23]:
cleaned_profile.rename(columns={"id": "customer_id"}, inplace=True)

In [24]:
cleaned_profile.describe()

,gender,age,income,member_days
count,14613.000000,14613.000000,14613.000000,14613.000000
mean,0.580579,54.393417,65435.707931,2653.991035
std,0.493481,17.400201,21633.524110,419.681797
min,0.000000,18.000000,30000.000000,2131.000000
25%,0.000000,42.000000,49000.000000,2339.000000
50%,1.000000,55.000000,64000.000000,2489.000000
75%,1.000000,66.000000,80000.000000,2929.000000
max,1.000000,101.000000,120000.000000,3954.000000


### Clean transcript

In [25]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [26]:
transcript["event"].unique()

array(['offer received', 'offer viewed', 'transaction', 'offer completed'],
      dtype=object)

In [27]:
offer_events = transcript[transcript["event"].str.contains("offer")]
transactions = transcript[transcript["event"] == "transaction"]

Create offer_events dataframe from transcript

In [28]:
offer_events["offer_id"] = offer_events["value"].apply(lambda x: x["offer id"] if "offer id" in x else x["offer_id"])

C:\Users\luong\AppData\Local\Temp\ipykernel_11024\310765619.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offer_events["offer_id"] = offer_events["value"].apply(lambda x: x["offer id"] if "offer id" in x else x["offer_id"])


In [29]:
events = offer_events["event"].apply(lambda x: x.split(' ')[1])

In [30]:
offer_events = pd.concat([offer_events, pd.get_dummies(events)], axis=1)

In [31]:
offer_events.drop(columns=["event", "value"], inplace=True)

In [32]:
offer_events.rename(columns={"person": "customer_id"}, inplace=True)

In [33]:
offer_events.head()

,customer_id,time,offer_id,completed,received,viewed
0,78afa995795e4d85b5d9ceeca43f5fef,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,False,True,False
1,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,False,True,False
2,e2127556f4f64592b11af22de27a7932,0,2906b810c7d4411798c6938adc9daaa5,False,True,False
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,fafdcd668e3743c1bb461111dcafc2a4,False,True,False
4,68617ca6246f4fbc85e91a2a49552598,0,4d5c57ea9a6940dd891ad53e9dbe8da0,False,True,False


Create transaction dataframe from transcript

In [34]:
transactions

,person,event,value,time
12654,02c083884c7d45b39cc68e1314fec56c,transaction,{'amount': 0.8300000000000001},0
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,{'amount': 34.56},0
12659,54890f68699049c2a04d415abc25e717,transaction,{'amount': 13.23},0
12670,b2f1cd155b864803ad8334cdf13c4bd2,transaction,{'amount': 19.51},0
12671,fe97aa22dd3e48c8b143116a8403dd52,transaction,{'amount': 18.97},0
...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714


In [35]:
transactions.drop(columns="event", inplace=True)

C:\Users\luong\AppData\Local\Temp\ipykernel_11024\1152326591.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions.drop(columns="event", inplace=True)


In [36]:
transactions["amount"] = transactions["value"].apply(lambda x: x['amount'])

C:\Users\luong\AppData\Local\Temp\ipykernel_11024\411274397.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions["amount"] = transactions["value"].apply(lambda x: x['amount'])


In [37]:
transactions.drop(columns="value", inplace=True)

C:\Users\luong\AppData\Local\Temp\ipykernel_11024\1848487943.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions.drop(columns="value", inplace=True)


In [38]:
transactions

,person,time,amount
12654,02c083884c7d45b39cc68e1314fec56c,0,0.83
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56
12659,54890f68699049c2a04d415abc25e717,0,13.23
12670,b2f1cd155b864803ad8334cdf13c4bd2,0,19.51
12671,fe97aa22dd3e48c8b143116a8403dd52,0,18.97
...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,714,1.59
306530,68213b08d99a4ae1b0dcb72aebd9aa35,714,9.53
306531,a00058cf10334a308c68e7631c529907,714,3.61
306532,76ddbd6576844afe811f1a3c0fbb5bec,714,3.53


### Comine transaction, profile and transcript data

In [39]:
cleaned_profile

,gender,age,customer_id,income,member_days
0,0,55,0610b486422d4921ae7d2bf64640c50b,112000.0,2507
1,0,75,78afa995795e4d85b5d9ceeca43f5fef,100000.0,2574
2,1,68,e2127556f4f64592b11af22de27a7932,70000.0,2222
3,1,65,389bc3fa690240e798340f5a15918d5c,53000.0,2298
4,1,58,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,2388
...,...,...,...,...,...
14608,0,45,6d5f3a774f3d4714ab0c092238f3a1d7,54000.0,2183
14609,1,61,2cb4f97358b841b9a9773a7aa05a9d77,72000.0,2144
14610,1,49,01d26f638c274aa0b965d24cefe3183f,73000.0,2677
14611,0,83,9dc1421481194dcd9400aec7c9ae6366,50000.0,3002


In [42]:
cleaned_portfolio.head()

,reward,difficulty,duration,offer_id,bogo,discount,informational,email,mobile,social,web
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,True,False,False,1,1,1,0
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,True,False,False,1,1,1,1
2,0,0,4,3f207df678b143eea3cee63160fa8bed,False,False,True,1,1,0,1
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,True,False,False,1,1,0,1
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,False,True,False,1,0,0,1


In [43]:
transactions

,person,time,amount
12654,02c083884c7d45b39cc68e1314fec56c,0,0.83
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56
12659,54890f68699049c2a04d415abc25e717,0,13.23
12670,b2f1cd155b864803ad8334cdf13c4bd2,0,19.51
12671,fe97aa22dd3e48c8b143116a8403dd52,0,18.97
...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,714,1.59
306530,68213b08d99a4ae1b0dcb72aebd9aa35,714,9.53
306531,a00058cf10334a308c68e7631c529907,714,3.61
306532,76ddbd6576844afe811f1a3c0fbb5bec,714,3.53


In [40]:
df = offer_events.merge(cleaned_profile, on="customer_id").merge(cleaned_portfolio, on="offer_id")

Select rows where user has completed the offer that they received

In [139]:
offer_taken_user = df.groupby(by=["customer_id", "offer_id"])["completed"].any()
customer_offer_ids = offer_taken_user[offer_taken_user == True].index

In [134]:
customer_offer_ids[0]

('0009655768c64bdeb2e877511632db8f', '2906b810c7d4411798c6938adc9daaa5')

In [124]:
taken_df = df.set_index(["customer_id", "offer_id"])

In [138]:
taken_df.loc[customer_offer_ids]

time  \
customer_id                      offer_id                                 
0009655768c64bdeb2e877511632db8f 2906b810c7d4411798c6938adc9daaa5   576   
                                 2906b810c7d4411798c6938adc9daaa5   576   
                                 f19421c1d4aa40978ebb69ca19b0e20d   408   
                                 f19421c1d4aa40978ebb69ca19b0e20d   414   
                                 f19421c1d4aa40978ebb69ca19b0e20d   456   
...                                                                 ...   
ffff82501cea40309d5fdd7edcca4a07 9b98b8c7a33c4b65b9aebfe6a799e6d9   504   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9   534   
                                 fafdcd668e3743c1bb461111dcafc2a4     0   
                                 fafdcd668e3743c1bb461111dcafc2a4     6   
                                 fafdcd668e3743c1bb461111dcafc2a4    60   

                                                                   completed  \
customer_id                      offer_id                                      
0009655768c64bdeb2e877511632db8f 2906b810c7d4411798c6938adc9daaa5      False   
                                 2906b810c7d4411798c6938adc9daaa5       True   
                                 f19421c1d4aa40978ebb69ca19b0e20d      False   
                                 f19421c1d4aa40978ebb69ca19b0e20d       True   
                                 f19421c1d4aa40978ebb69ca19b0e20d      False   
...                                                                      ...   
ffff82501cea40309d5fdd7edcca4a07 9b98b8c7a33c4b65b9aebfe6a799e6d9       True   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9      False   
                                 fafdcd668e3743c1bb461111dcafc2a4      False   
                                 fafdcd668e3743c1bb461111dcafc2a4      False   
                                 fafdcd668e3743c1bb461111dcafc2a4       True   

                                                                   received  \
customer_id                      offer_id                                     
0009655768c64bdeb2e877511632db8f 2906b810c7d4411798c6938adc9daaa5      True   
                                 2906b810c7d4411798c6938adc9daaa5     False   
                                 f19421c1d4aa40978ebb69ca19b0e20d      True   
                                 f19421c1d4aa40978ebb69ca19b0e20d     False   
                                 f19421c1d4aa40978ebb69ca19b0e20d     False   
...                                                                     ...   
ffff82501cea40309d5fdd7edcca4a07 9b98b8c7a33c4b65b9aebfe6a799e6d9     False   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9     False   
                                 fafdcd668e3743c1bb461111dcafc2a4      True   
                                 fafdcd668e3743c1bb461111dcafc2a4     False   
                                 fafdcd668e3743c1bb461111dcafc2a4     False   

                                                                   viewed  \
customer_id                      offer_id                                   
0009655768c64bdeb2e877511632db8f 2906b810c7d4411798c6938adc9daaa5   False   
                                 2906b810c7d4411798c6938adc9daaa5   False   
                                 f19421c1d4aa40978ebb69ca19b0e20d   False   
                                 f19421c1d4aa40978ebb69ca19b0e20d   False   
                                 f19421c1d4aa40978ebb69ca19b0e20d    True   
...                                                                   ...   
ffff82501cea40309d5fdd7edcca4a07 9b98b8c7a33c4b65b9aebfe6a799e6d9   False   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9    True   
                                 fafdcd668e3743c1bb461111dcafc2a4   False   
                                 fafdcd668e3743c1bb461111dcafc2a4    True   
                                 fafdcd668e3743c1bb461111dcafc2a4   False   

                    

In [127]:
taken_df.head()

time  \
customer_id                      offer_id                                 
78afa995795e4d85b5d9ceeca43f5fef 9b98b8c7a33c4b65b9aebfe6a799e6d9     0   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9     6   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9   132   
e2127556f4f64592b11af22de27a7932 9b98b8c7a33c4b65b9aebfe6a799e6d9   408   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9   420   

                                                                   completed  \
customer_id                      offer_id                                      
78afa995795e4d85b5d9ceeca43f5fef 9b98b8c7a33c4b65b9aebfe6a799e6d9      False   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9      False   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9       True   
e2127556f4f64592b11af22de27a7932 9b98b8c7a33c4b65b9aebfe6a799e6d9      False   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9      False   

                                                                   received  \
customer_id                      offer_id                                     
78afa995795e4d85b5d9ceeca43f5fef 9b98b8c7a33c4b65b9aebfe6a799e6d9      True   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9     False   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9     False   
e2127556f4f64592b11af22de27a7932 9b98b8c7a33c4b65b9aebfe6a799e6d9      True   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9     False   

                                                                   viewed  \
customer_id                      offer_id                                   
78afa995795e4d85b5d9ceeca43f5fef 9b98b8c7a33c4b65b9aebfe6a799e6d9   False   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9    True   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9   False   
e2127556f4f64592b11af22de27a7932 9b98b8c7a33c4b65b9aebfe6a799e6d9   False   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9    True   

                                                                   gender  \
customer_id                      offer_id                                   
78afa995795e4d85b5d9ceeca43f5fef 9b98b8c7a33c4b65b9aebfe6a799e6d9       0   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9       0   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9       0   
e2127556f4f64592b11af22de27a7932 9b98b8c7a33c4b65b9aebfe6a799e6d9       1   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9       1   

                                                                   age  \
customer_id                      offer_id                                
78afa995795e4d85b5d9ceeca43f5fef 9b98b8c7a33c4b65b9aebfe6a799e6d9   75   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9   75   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9   75   
e2127556f4f64592b11af22de27a7932 9b98b8c7a33c4b65b9aebfe6a799e6d9   68   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9   68   

                                                                     income  \
customer_id                      offer_id                                     
78afa995795e4d85b5d9ceeca43f5fef 9b98b8c7a33c4b65b9aebfe6a799e6d9  100000.0   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9  100000.0   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9  100000.0   
e2127556f4f64592b11af22de27a7932 9b98b8c7a33c4b65b9aebfe6a799e6d9   70000.0   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9   70000.0   

                                                                   member_days  \
customer_id                      offer_id                                        
78afa995795e4d85b5d9ceeca43f5fef 9b98b8c7a33c4b65b9aebfe6a799e6d9         2574   
                           

In [121]:
taken_df = df[(df["customer_id"].isin(customer_ids["customer_id"])) & (df["offer_id"].isin(customer_ids["offer_id"]))]

In [123]:
taken_df[taken_df["customer_id"] == "0020c2b971eb4e9188eac86d93036a77"]

,customer_id,time,offer_id,completed,received,viewed,gender,age,income,member_days,reward,difficulty,duration,bogo,discount,informational,email,mobile,social,web
28205,0020c2b971eb4e9188eac86d93036a77,168,ae264e3637204a6fb9bb56bc8210ddfd,False,True,False,0,59,90000.0,3005,10,10,7,True,False,False,1,1,1,0
85137,0020c2b971eb4e9188eac86d93036a77,0,fafdcd668e3743c1bb461111dcafc2a4,False,True,False,0,59,90000.0,3005,2,10,10,False,True,False,1,1,1,1
85138,0020c2b971eb4e9188eac86d93036a77,12,fafdcd668e3743c1bb461111dcafc2a4,False,False,True,0,59,90000.0,3005,2,10,10,False,True,False,1,1,1,1
85139,0020c2b971eb4e9188eac86d93036a77,54,fafdcd668e3743c1bb461111dcafc2a4,True,False,False,0,59,90000.0,3005,2,10,10,False,True,False,1,1,1,1
85140,0020c2b971eb4e9188eac86d93036a77,336,fafdcd668e3743c1bb461111dcafc2a4,False,True,False,0,59,90000.0,3005,2,10,10,False,True,False,1,1,1,1
85141,0020c2b971eb4e9188eac86d93036a77,510,fafdcd668e3743c1bb461111dcafc2a4,True,False,False,0,59,90000.0,3005,2,10,10,False,True,False,1,1,1,1
132530,0020c2b971eb4e9188eac86d93036a77,408,4d5c57ea9a6940dd891ad53e9dbe8da0,False,True,False,0,59,90000.0,3005,10,10,5,True,False,False,1,1,1,1
132531,0020c2b971eb4e9188eac86d93036a77,426,4d5c57ea9a6940dd891ad53e9dbe8da0,False,False,True,0,59,90000.0,3005,10,10,5,True,False,False,1,1,1,1
132532,0020c2b971eb4e9188eac86d93036a77,510,4d5c57ea9a6940dd891ad53e9dbe8da0,True,False,False,0,59,90000.0,3005,10,10,5,True,False,False,1,1,1,1


In [122]:
taken_df.groupby(by=["customer_id", "offer_id"])["time"].count()

customer_id                       offer_id                        
0009655768c64bdeb2e877511632db8f  2906b810c7d4411798c6938adc9daaa5    2
                                  f19421c1d4aa40978ebb69ca19b0e20d    3
                                  fafdcd668e3743c1bb461111dcafc2a4    3
0020c2b971eb4e9188eac86d93036a77  4d5c57ea9a6940dd891ad53e9dbe8da0    3
                                  ae264e3637204a6fb9bb56bc8210ddfd    1
                                                                     ..
fffad4f4828548d1b5583907f2e9906b  f19421c1d4aa40978ebb69ca19b0e20d    6
ffff82501cea40309d5fdd7edcca4a07  0b1e1539f2cc45b7b9fa7c272da2e1d7    3
                                  2906b810c7d4411798c6938adc9daaa5    9
                                  9b98b8c7a33c4b65b9aebfe6a799e6d9    3
                                  fafdcd668e3743c1bb461111dcafc2a4    3
Name: time, Length: 35981, dtype: int64

In [110]:
df[(df["customer_id"] == "0020c2b971eb4e9188eac86d93036a77") & (df["offer_id"] == "ae264e3637204a6fb9bb56bc8210ddfd")]

,customer_id,time,offer_id,completed,received,viewed,gender,age,income,member_days,reward,difficulty,duration,bogo,discount,informational,email,mobile,social,web
28205,0020c2b971eb4e9188eac86d93036a77,168,ae264e3637204a6fb9bb56bc8210ddfd,False,True,False,0,59,90000.0,3005,10,10,7,True,False,False,1,1,1,0
